In [ ]:
import streamlit as st
import joblib
import yfinance as yf
import numpy as np
import pandas as pd
import os
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler, StandardScaler

st.set_page_config(page_title="NIFTY50 AI Predictor", layout="centered")

MODEL_FILE = "trained_models.joblib"

st.title(" Stock Price Prediction Tool")
st.write("Live AI-powered stock price predictions.")

@st.cache_resource
def load_models(path=MODEL_FILE):
    if os.path.exists(path):
        return joblib.load(path)
    return None

models = load_models()

if models is None:
    st.error(f"⚠️ Could not find '{MODEL_FILE}'. Please move it to this folder.")
    st.stop()

@st.cache_data
def fetch_data(sym):
    return yf.download(sym, period="10y", interval="1d", progress=False)

default_symbol = "^NSEI"
with st.spinner(f"Analyzing market data..."):
    data = fetch_data(default_symbol)

if data is None or data.empty:
    st.error("❌ No data found.")
    st.stop()

try:
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)
    close_data = data["Close"]
    if isinstance(close_data, pd.DataFrame):
        close_data = close_data.iloc[:, 0]
    close_data = close_data.dropna()
    
    min_date = close_data.index[100].date() 
    max_date_available = close_data.index[-1].date()
    next_trading_day = max_date_available + pd.Timedelta(days=1)
except Exception as e:
    st.error(f"Data Error: {e}")
    st.stop()

with st.sidebar:
    st.header("Settings")
    symbol = st.text_input("Ticker Symbol", value="^NSEI")

    all_keys = sorted(list(models.keys()))
    filtered_keys = [k for k in all_keys if "knn" in k.lower() and "90" in k]
    
    if filtered_keys:
        model_keys = filtered_keys
        st.success(f"Loaded {len(model_keys)} KNN Models")
    else:
        model_keys = all_keys
        st.info("Showing all available models")

    selected_key = st.selectbox("Choose Model", model_keys)

    st.markdown("---")
    st.header("📅 Prediction Date")
    target_date = st.date_input(
        "Predict Price For:", 
        value=next_trading_day, 
        min_value=min_date, 
        max_value=next_trading_day
    )
    
    is_forecast = target_date > max_date_available
    mode_text = "Future Forecast" if is_forecast else "Historical Check"
    st.caption(f"Mode: {mode_text}")

    st.markdown("---")
    with st.expander("🛠️ Advanced Settings"):
        scaler_option = st.radio(
            "Data Transformation:", 
            ["MinMaxScaler", "StandardScaler", "Log Price", "None (Raw Data)"],
            index=3 
        )

window_size = 90
if "_" in selected_key:
    try:
        parts = selected_key.split("_")
        if parts[-1].isdigit():
            window_size = int(parts[-1])
    except:
        pass

try:
    actual_price_on_date = None
    
    if is_forecast:
        
        raw_window = close_data.values[-window_size:].reshape(-1, 1)
        current_price = float(close_data.iloc[-1])
        date_label = "Last Close"
    else:
        search_ts = pd.Timestamp(target_date)
        
        try:
            idx_loc = close_data.index.get_indexer([search_ts], method='pad')[0]
        except:
            st.error("Invalid date selected.")
            st.stop()

        if idx_loc < window_size:
            st.error("Not enough history before this date to make a prediction.")
            st.stop()

        raw_window = close_data.values[idx_loc-window_size : idx_loc].reshape(-1, 1)
        
        current_price = float(close_data.iloc[idx_loc-1])
        actual_price_on_date = float(close_data.iloc[idx_loc]) 
        date_label = f"Close on {close_data.index[idx_loc-1].date()}"

except Exception as e:
    st.error(f"Date Processing Error: {e}")
    st.stop()

try:
    input_data = raw_window
    scaler = None
    
    if "MinMax" in scaler_option:
        scaler = MinMaxScaler(feature_range=(0,1))
        scaler.fit(close_data.values.reshape(-1, 1)) 
        input_data = scaler.transform(raw_window)
    elif "Standard" in scaler_option:
        scaler = StandardScaler()
        scaler.fit(close_data.values.reshape(-1, 1))
        input_data = scaler.transform(raw_window)
    elif "Log" in scaler_option:
        input_data = np.log(raw_window)

    model_entry = models[selected_key]
    model_obj = model_entry.get("model", model_entry)
    
    raw_pred = None
    try:
        model_input = input_data.reshape(1, -1) 
        raw_pred = model_obj.predict(model_input)
    except:
        model_input = input_data.reshape(1, window_size, 1) 
        raw_pred = model_obj.predict(model_input)

    pred_value = float(np.array(raw_pred).ravel()[0])

    final_prediction = pred_value
    if "MinMax" in scaler_option or "Standard" in scaler_option:
        final_prediction = float(scaler.inverse_transform([[pred_value]])[0][0])
    elif "Log" in scaler_option:
        final_prediction = float(np.exp(pred_value))

    col1, col2, col3 = st.columns(3)
    
    with col1:
        st.subheader("Previous Close")
        st.metric(label=date_label, value=f"₹{current_price:,.2f}")
    
    with col2:
        st.subheader("AI Prediction")
        delta = final_prediction - current_price
        st.metric(
            label=f"Forecast for {target_date}", 
            value=f"₹{final_prediction:,.2f}", 
            delta=f"{delta:,.2f}"
        )

    with col3:
        if actual_price_on_date:
            st.subheader("Actual Price")
            accuracy = 100 - (abs(actual_price_on_date - final_prediction) / actual_price_on_date * 100)
            diff = final_prediction - actual_price_on_date
            
            st.metric(
                label="What actually happened", 
                value=f"₹{actual_price_on_date:,.2f}",
                delta=f"Error: {diff:,.2f}",
                delta_color="off"
            )
            st.caption(f"Accuracy: {accuracy:.1f}%")
        else:
            st.subheader("Actual Price")
            st.write("Unknown (Future)")

    st.write("---")
    st.subheader("Context Chart")
    
    target_ts = pd.Timestamp(target_date)
    start_plot = target_ts - pd.Timedelta(days=90)
    end_plot = target_ts + pd.Timedelta(days=30)
    
    
    chart_data = close_data[start_plot:end_plot] if is_forecast else close_data[start_plot:end_plot]
    if len(chart_data) == 0:
        chart_data = close_data[-90:] 
    
    fig = go.Figure()
    
   
    fig.add_trace(go.Scatter(
        x=chart_data.index, y=chart_data, mode='lines', name='Actual History',
        line=dict(color='#00B4D8', width=2)
    ))
    
   
    fig.add_trace(go.Scatter(
        x=[target_ts],
        y=[final_prediction],
        mode='markers', name='AI Prediction',
        marker=dict(color='#FF4B4B', size=12, symbol='star')
    ))

    fig.update_layout(
        template="plotly_dark", 
        paper_bgcolor='rgba(0,0,0,0)', 
        plot_bgcolor='rgba(0,0,0,0)', 
        height=450,
        title=f"Market Context around {target_date}"
    )
    st.plotly_chart(fig, use_container_width=True)

except Exception as e:
    st.error(f"Prediction Error: {e}")